### Load Damagescanner package

In [5]:
import os
import rasterio
import pygeos
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
#from damagescanner.core import VectorScanner 

data_path = '..'

### Read input data

In [6]:
inun_map = os.path.join(data_path,'data','inundation','inundation_map.tif')
landuse_map = os.path.join(data_path,'data','landuse','landuse_map.tif')
curve_path = os.path.join(data_path,'data','curves','curves.csv')
maxdam_path = os.path.join(data_path,'data','curves','maxdam.csv')

### Run the RasterScanner

In [177]:
%%time
# Return dataframe with losses per land-use clas
damage_df = RasterScanner(landuse_map,inun_map,curve_path,maxdam_path)[0]

CPU times: total: 906 ms
Wall time: 927 ms


In [197]:
landuse = xr.open_dataset(landuse_map, engine="rasterio")
landuse.rio.write_crs(28992,inplace=True)
landuse = landuse.rio.reproject("EPSG:3035")

In [212]:
with rasterio.open(landuse_map) as src:
    landuse = src.read()[0, :, :]
    transform = src.transform

In [215]:
landuse.shape

(2269, 3476)

In [198]:
landuse['band_data'].to_numpy().shape

(1, 2486, 3614)

In [199]:
hazard_file = 'C3S_fp_era5_sd_20131028_95_0_Christian_v1.nc'
hazard = xr.open_dataset(hazard_file)
hazard.rio.write_crs(4326, inplace=True)
hazard = hazard.rename({'Latitude': 'y','Longitude': 'x'})
hazard.rio.set_spatial_dims(x_dim="x",y_dim="y", inplace=True)

<xarray.Dataset>
Dimensions:      (x: 3944, y: 2432, z: 1)
Coordinates:
  * x            (x) float64 -25.36 -25.34 -25.32 -25.31 ... 40.32 40.34 40.36
  * y            (y) float64 70.35 70.33 70.32 70.3 ... 29.88 29.87 29.85 29.83
  * z            (z) int32 1
    spatial_ref  int32 0
Data variables:
    FX           (z, y, x) float32 ...
Attributes:
    crs_format:       PROJ.4
    Conventions:      CF-1.4
    created_by:       R, packages ncdf4 and raster (version 2.6-7)
    date:             2019-07-03 12:23:39
    title:            C3S operational wind storm
    institution:      Royal Netherlands Meteorological Institute (KNMI)
    contact:          Please contact Copernicus User Support on the Copernicu...
    licence:          Licence to use Copernicus products
    version:          v1
    product_version:  1

In [200]:
hazard.dims['x'],hazard.dims['y']

(3944, 2432)

In [201]:
min_lon = landuse.x.min().to_dict()['data']
min_lat = landuse.y.min().to_dict()['data']
max_lon = landuse.x.max().to_dict()['data']
max_lat = landuse.y.max().to_dict()['data']

#subset = hazard.rio.clip_box(minx=min_lon, miny=min_lat, maxx=max_lon, maxy=max_lat)
#subset = subset.rio.reproject("EPSG:3035",resolution=5)

In [202]:
area = gpd.GeoDataFrame([pygeos.box(min_lon,min_lat,max_lon, max_lat)],columns=['geometry'])
area.crs = 'epsg:3035'
area = area.to_crs('epsg:4326')

In [210]:
windstorm = hazard.rio.clip(area.geometry.values, area.crs)
windstorm = windstorm.rio.reproject("EPSG:3035",resolution=5)
windstorm = windstorm.rio.clip(area.geometry.values, area.crs)

In [211]:
landuse.rio.reproject_match(windstorm)

<xarray.Dataset>
Dimensions:      (x: 3578, y: 2485, band: 1)
Coordinates:
  * x            (x) float64 4.034e+06 4.034e+06 ... 4.052e+06 4.052e+06
  * y            (y) float64 3.286e+06 3.286e+06 ... 3.274e+06 3.274e+06
  * band         (band) int32 1
    spatial_ref  int32 0
Data variables:
    band_data    (band, y, x) float64 nan nan nan nan nan ... nan nan nan nan

In [207]:
landuse

<xarray.Dataset>
Dimensions:      (x: 3614, y: 2486, band: 1)
Coordinates:
  * x            (x) float64 4.034e+06 4.034e+06 ... 4.052e+06 4.052e+06
  * y            (y) float64 3.286e+06 3.286e+06 ... 3.274e+06 3.274e+06
  * band         (band) int32 1
    spatial_ref  int32 0
Data variables:
    band_data    (band, y, x) float64 nan nan nan nan nan ... nan nan nan nan